<a href="https://colab.research.google.com/github/omoreva/kaggle_m5/blob/master/10_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tsfresh

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime  
from datetime import timedelta  
import gc
import pickle
from sklearn import preprocessing, metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters, ComprehensiveFCParameters

In [0]:
print('Reading files...')
#data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'calendar.csv')
data_folder = '/content/drive/My Drive/Kaggle_M5/01_preprocessed_data/'
X_train = pd.read_pickle(data_folder + 'X_train_WI_2.pkl')
X_val = pd.read_pickle(data_folder + 'X_val_WI_2.pkl')
y_train = X_train['demand']
y_val = X_val['demand']


Reading files...


In [0]:
X_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,weekday,month,event_name_1,event_type_1,year,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,start_date,days_from_start,start_date_from_start,sell_price,lag_28,lag_29,lag_30,lag_31,lag_32,lag_33,lag_34,lag_35,rmean_28_3,rmean_28_7,ADI,CV2,demand_type,avg_weekday_demand,avg_demand,max_demand,std_demand,zero_seq_len,non_zero_seq_len,quantile025_week,max_weekday_demand,avg_price,cheaper_than_usual
439578,HOBBIES_1_008_WI_2_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,WI_2,WI,d_36,0,2011-03-05,Saturday,3,no event,no event,2011,no event,no event,1,1,1,2011-01-29,35,0,0.419922,0.0,0.0,1.0,0.0,12.0,13.0,3.0,2.0,0.333252,4.144531,1.546875,0.949219,lumpy,3.437500,3.001953,51,4.261719,182,32,0.0,48,0.476318,True
439579,HOBBIES_1_009_WI_2_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,WI_2,WI,d_36,1,2011-03-05,Saturday,3,no event,no event,2011,no event,no event,1,1,1,2011-01-29,35,0,1.599609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.000000,0.000000,5.890625,0.707520,lumpy,0.335938,0.262207,12,0.789062,190,5,0.0,12,1.746094,True
439580,HOBBIES_1_010_WI_2_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,WI_2,WI,d_36,0,2011-03-05,Saturday,3,no event,no event,2011,no event,no event,1,1,1,2011-01-29,35,0,3.169922,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000,0.571289,3.023438,0.206421,intermittent,0.494385,0.425293,5,0.692383,200,8,0.0,4,2.970703,False
439581,HOBBIES_1_012_WI_2_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,WI_2,WI,d_36,1,2011-03-05,Saturday,3,no event,no event,2011,no event,no event,1,1,1,2011-01-29,35,0,6.578125,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.666504,0.285645,4.882812,0.109985,intermittent,0.313232,0.231934,4,0.487549,36,5,0.0,3,6.480469,False
439583,HOBBIES_1_016_WI_2_validation,HOBBIES_1_016,HOBBIES_1,HOBBIES,WI_2,WI,d_36,1,2011-03-05,Saturday,3,no event,no event,2011,no event,no event,1,1,1,2011-01-29,35,0,0.830078,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.666504,0.714355,2.326172,0.886719,lumpy,1.301758,1.319336,25,2.429688,95,13,0.0,24,0.722656,False


In [0]:
my_parameters = {
 'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
  {'f_agg': 'median', 'maxlag': 40},
  {'f_agg': 'var', 'maxlag': 40}],
  'ar_coefficient': [{'coeff': 3, 'k': 10}],
   'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
  {'num_segments': 10, 'segment_focus': 1},
  {'num_segments': 10, 'segment_focus': 2},
  {'num_segments': 10, 'segment_focus': 3},
  {'num_segments': 10, 'segment_focus': 4},
  {'num_segments': 10, 'segment_focus': 5},
  {'num_segments': 10, 'segment_focus': 6},
  {'num_segments': 10, 'segment_focus': 7},
  {'num_segments': 10, 'segment_focus': 8},
  {'num_segments': 10, 'segment_focus': 9}],
 'fft_coefficient': [{'attr': 'angle', 'coeff': 1},
],
}

In [0]:
store_demand = X_train[['item_id', 'date', 'demand']]

In [0]:
store_demand_fe = extract_features(store_demand, 
                     column_id='item_id', column_sort='date',
                     default_fc_parameters=my_parameters)

Feature Extraction: 100%|██████████| 10/10 [00:17<00:00,  1.79s/it]


In [0]:
store_demand_fe = store_demand_fe.reset_index()
store_demand_fe = store_demand_fe.rename(columns={"id": "item_id"})
store_demand_fe.head()

X_train = X_train.merge(store_demand_fe, on = ['item_id'], how = 'left')
X_val = X_val.merge(store_demand_fe, on = ['item_id'], how = 'left')



In [0]:
not_features = ['d', 'id', 'item_id', 'store_id', 'dept_id',
                'state_id',  'date', 'start_date', 
                'snap_CA', 	'snap_TX', 'event_type_1', 'event_name_2', 'event_type_2',
                'demand_type']
features = X_train.columns[~X_train.columns.isin(not_features) ]
X_train = X_train[features]
X_val = X_val[features]


In [0]:
features

Index(['cat_id', 'demand', 'weekday', 'month', 'event_name_1', 'year',
       'snap_WI', 'days_from_start', 'start_date_from_start', 'sell_price',
       'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34',
       'lag_35', 'rmean_28_3', 'rmean_28_7', 'ADI', 'CV2',
       'avg_weekday_demand', 'avg_demand', 'max_demand', 'std_demand',
       'zero_seq_len', 'non_zero_seq_len', 'quantile025_week',
       'max_weekday_demand', 'avg_price', 'cheaper_than_usual',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3',
       'demand__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
       'demand__energy_ratio_by_chunks__num_segments_10__segment_focus_1',
       'demand__energy_ratio_by_chunks__num_segments_10__segment_focus_2',
       'demand__energy_ratio_by_chunks__num_segments_10_

In [0]:
for column in X_val.columns:
 if X_val[column].dtype.name == 'category':
  dummyCols=pd.get_dummies(X_val[column])
  X_val=X_val.join(dummyCols)
  del X_val[column]

In [0]:
for column in X_train.columns:
 if X_train[column].dtype.name == 'category':
  dummyCols=pd.get_dummies(X_train[column])
  X_train=X_train.join(dummyCols)
  del X_train[column]

In [0]:
X_val = X_val.fillna(0)

In [0]:
X_train = X_train.dropna()

In [0]:
y_train = X_train['demand']
y_val = X_val['demand']
del X_train['demand']
del X_val['demand']

In [0]:
for feature in X_train.columns:
    print("{}: {}".format(feature, X_train[feature].corr(y_train)))

month: -0.005331694632323964
year: 0.00602476206448677
snap_WI: 0.04677626410824542
days_from_start: 0.004378053822540438
start_date_from_start: -0.02268596489262877
sell_price: -0.16807703617770417
lag_28: 0.5784407577671664
lag_29: 0.5661573644994532
lag_30: 0.5651533000667964
lag_31: 0.563769764621623
lag_32: 0.5533464205712468
lag_33: 0.5491572710858105
lag_34: 0.5528994652686361
lag_35: 0.5484823804133633
rmean_28_3: 0.6383461138395076
rmean_28_7: 0.6679982503163494
ADI: -0.1847997646336234
CV2: 0.17971717721806027
avg_weekday_demand: 0.6610092145325063
avg_demand: 0.6554178429515397
max_demand: 0.5434951921252689
std_demand: 0.6064130928759729
zero_seq_len: -0.07732519814595432
non_zero_seq_len: 0.5170988900343764
quantile025_week: 0.4606684628598245
max_weekday_demand: 0.5691419770604416
avg_price: -0.1678535454624901
cheaper_than_usual: 0.04361777909577033
demand__agg_autocorrelation__f_agg_"mean"__maxlag_40: 0.2918033404843618
demand__agg_autocorrelation__f_agg_"median"__maxla

In [0]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_val)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_val, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_val, y_pred))


Coefficients: 
 [ 8.47351962e-02  1.07707565e+00  2.82887878e-01 -2.86382925e-03
 -2.66701972e-05 -5.92892101e-02 -1.19584483e+00 -1.24665075e+00
 -1.23497397e+00 -4.15449410e-01 -4.37846447e-01 -4.43350881e-01
 -4.33852294e-01  4.21568006e-02  2.46062636e+00  3.39192315e+00
  1.78711683e-04 -1.76329607e-02  9.08509599e-01 -3.88399019e-01
  6.12488252e-04 -8.17231742e-03 -1.38674177e-05 -6.56565977e-05
 -5.31022942e-03  2.61887215e-04  5.85676545e-02  6.31020074e-02
 -2.79107977e-01  3.11375004e-01  6.52241564e-01  6.08269243e-03
 -8.85914518e-02 -5.56059142e-02 -2.45791186e-02 -2.38840793e-02
 -5.15861299e-02 -4.83335126e-02 -1.97024285e-02 -5.96211073e-02
 -3.48046310e-02  5.98075538e-02 -8.97012294e-07 -4.12316369e-03
  5.14840543e-03 -1.02524174e-03  3.77482566e-03 -1.79903700e-02
 -6.84239224e-03  1.43416489e-02  1.86103258e-02 -7.27800681e-03
 -4.61603133e-03  1.78435379e-01 -9.85943371e-01 -9.88021592e-02
  1.32312614e-01 -3.11670223e-01  1.28213590e-01  1.70151665e-01
  2.84187

In [0]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_val)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_val, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_val, y_pred))

In [0]:
X_train[X_train.isnull().any(axis=1)]

,month,year,snap_WI,days_from_start,start_date_from_start,sell_price,lag_28,lag_29,lag_30,lag_31,lag_32,lag_33,lag_34,lag_35,rmean_28_3,rmean_28_7,ADI,CV2,avg_weekday_demand,avg_demand,max_demand,std_demand,zero_seq_len,non_zero_seq_len,quantile025_week,max_weekday_demand,avg_price,cheaper_than_usual,index,"demand__agg_autocorrelation__f_agg_""mean""__maxlag_40_x","demand__agg_autocorrelation__f_agg_""median""__maxlag_40_x","demand__agg_autocorrelation__f_agg_""var""__maxlag_40_x",demand__ar_coefficient__k_10__coeff_3_x,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_0_x,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_1_x,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_2_x,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_3_x,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_4_x,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_5_x,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_6_x,...,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_5_y,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_6_y,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_7_y,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_8_y,demand__energy_ratio_by_chunks__num_segments_10__segment_focus_9_y,"demand__fft_coefficient__coeff_1__attr_""angle""_y",FOODS,HOBBIES,HOUSEHOLD,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsEnd,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,no event
